<a href="https://colab.research.google.com/github/grayzerguy/02-homework-solution/blob/master/ZARA_XML_TO_EXCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

עבור משלוחים זארה


In [ ]:
from google.colab import files
import xml.etree.ElementTree as ET
import pandas as pd
import os

# 📤 העלאת כמה קבצים
uploaded = files.upload()

# 🧩 פונקציה שמעבדת קובץ XML
def process_xml(xml_content, source_name):
    root = ET.fromstring(xml_content)
    ns = {'ns': 'http://www.inditex.com/Schemas/Ecommerce/ItxShipmentRequest'}
    rows = []

    for order in root.findall('.//ns:Order', ns):
        order_data = order.attrib
        for parcel in order.find('ns:Parcels', ns).findall('ns:Parcel', ns):
            parcel_data = parcel.attrib
            row = {
                "SourceName": source_name,
                "OrderNumber": order_data.get("OrderNumber", ""),
                "CustomerName": order_data.get("CustomerName", ""),
                "CustomerSurname": order_data.get("CustomerSurname", ""),
                "Phone": order_data.get("PhoneNumber1", ""),
                "City": order_data.get("City", ""),
                "AddressLine1": order_data.get("AddressLine1", ""),
                "AddressLine2": order_data.get("AddressLine2", ""),
                "ParcelRef": parcel_data.get("InditexBoxReference", ""),
                "CourierRef": parcel_data.get("CourierBoxReference", ""),
                "Weight": parcel_data.get("Weight", ""),
                "PackageType": parcel_data.get("PackageType", "")
            }
            rows.append(row)

    return rows

# 🔄 עיבוד כל הקבצים
all_rows = []

for file_name, content in uploaded.items():
    # כאן תוכלי להגדיר שם מותאם לכל קובץ:
    # לדוגמה: לפי מיפוי ידני או פשוט לפי שם קובץ בלי סיומת
    source_name = os.path.splitext(file_name)[0]
    rows = process_xml(content.decode('utf-8'), source_name)
    all_rows.extend(rows)

# 🧾 יצירת DataFrame ו-Excel
df = pd.DataFrame(all_rows)
output_file = "combined_parcels.xlsx"
df.to_excel(output_file, index=False)

# 📥 הורדה למחשב
files.download(output_file)


עבור החזרות זארה

In [ ]:
from google.colab import files
import xml.etree.ElementTree as ET
import pandas as pd
import os

# 📤 העלאת קבצים
uploaded = files.upload()

# 🧩 פונקציה שמעבדת XML של החזרות
def process_return_xml(xml_content, source_name):
    root = ET.fromstring(xml_content)
    ns = {'ns': 'http://www.inditex.com/Schemas/Ecommerce/ItxReturnRequest'}
    rows = []

    for ret in root.findall('ns:ReturnRequest', ns):
        order_data = {
            "SourceName": source_name,
            "OrderNumber": ret.findtext('ns:OrderNumber', default='', namespaces=ns),
            "CustomerOrderNumber": ret.findtext('ns:CustomerOrderNumber', default='', namespaces=ns),
            "ReturnIdentifier": ret.findtext('ns:ReturnIdentifier', default='', namespaces=ns),
            "ReturnBoxIdentifier": ret.findtext('ns:ReturnBoxIdentifier', default='', namespaces=ns),
            "ReturnRequestDate": ret.findtext('ns:ReturnRequestDate', default='', namespaces=ns),
            "CustomerReturnRequestType": ret.findtext('ns:CustomerReturnRequestType', default='', namespaces=ns),
            "Brand": ret.findtext('ns:Brand', default='', namespaces=ns),
        }

        # כתובת איסוף
        pickup = ret.find('ns:PickUpAddress', ns)
        if pickup is not None:
            order_data.update({
                "CustomerName": pickup.attrib.get("CustomerName", ""),
                "CustomerSurname": pickup.attrib.get("CustomerSurName", ""),
                "PickupCity": pickup.attrib.get("City", ""),
                "PickupAddress": pickup.attrib.get("AddressLine1", ""),
                "Phone": pickup.attrib.get("PhoneNumber1", ""),
                "Email": pickup.attrib.get("Email", "")
            })

        # פריטים
        lines = ret.find('ns:Lines', ns)
        if lines is not None:
            for line in lines.findall('ns:Line', ns):
                row = order_data.copy()
                row.update({
                    "SKU": line.attrib.get("SKU", ""),
                    "Quantity": line.attrib.get("Quantity", "")
                })
                rows.append(row)

    return rows

# 🔄 עיבוד כל הקבצים
all_rows = []

for file_name, content in uploaded.items():
    source_name = os.path.splitext(file_name)[0]
    rows = process_return_xml(content.decode('utf-8'), source_name)
    all_rows.extend(rows)

# 📄 יצירת Excel
df = pd.DataFrame(all_rows)
output_file = "returns_combined.xlsx"
df.to_excel(output_file, index=False)

# 📥 הורדה
files.download(output_file)
/content/ReturnRequest202505140341380987.xml

In [ ]:
import os

# נתיב לתיקייה שתרצה לנקות
folder_path = '/content/sample_data'

# בדיקה אם התיקייה קיימת
if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):  # נוודא שזה קובץ, לא תיקייה
            os.remove(file_path)
            print(f"✅ Deleted: {file_path}")
    print("🧹 All files deleted successfully.")
else:
    print(f"❌ Folder not found: {folder_path}")



In [ ]:
from google.colab import files
import xml.etree.ElementTree as ET
import pandas as pd
import os

# 📤 העלאת קבצים
uploaded = files.upload()
all_rows = []

def process_tracking(xml_content, source_name):
    root = ET.fromstring(xml_content)
    ns = {'ns': 'http://www.inditex.com/Schemas/Ecommerce/ItxShipmentTracking'}
    rows = []

    header = root.find('ns:Header', ns)
    brand = header.attrib.get("Brand", "") if header is not None else ""
    generation_date = header.attrib.get("GenerationDate", "") if header is not None else ""

    for tu in root.findall('.//ns:TrackingUpdate', ns):
        row = {
            "SourceName": source_name,
            "Brand": brand,
            "GenerationDate": generation_date,
            "ChangeStatusDate": tu.attrib.get("ChangeStatusDate", ""),
            "InditexBoxReference": tu.attrib.get("InditexBoxReference", ""),
            "CourierCode": tu.attrib.get("CourierCode", ""),
            "CourierDesc": tu.attrib.get("CourierDesc", ""),
            "TrackingType": tu.attrib.get("TrackingType", ""),
            "Url": tu.attrib.get("Url", ""),
            "CourierStatusCode": tu.attrib.get("CourierStatusCode", "")
        }
        rows.append(row)
    return rows

# עיבוד כל הקבצים
for file_name, content in uploaded.items():
    source_name = os.path.splitext(file_name)[0]
    xml_text = content.decode('utf-8')
    try:
        rows = process_tracking(xml_text, source_name)
        all_rows.extend(rows)
    except Exception as e:
        print(f"❌ שגיאה בעיבוד {file_name}: {e}")

# יצירת קובץ אקסל
df = pd.DataFrame(all_rows)
output_file = "shipment_tracking.xlsx"
df.to_excel(output_file, index=False)

# הורדה
files.download(output_file)


ZARA JSON FROM MONGODB


In [ ]:
from google.colab import files
import json
import pandas as pd

# 📤 העלאת קובץ JSON
uploaded = files.upload()
filename = next(iter(uploaded))

# 📖 קריאת הקובץ
with open(filename, encoding='utf-8') as f:
    data = json.load(f)

# 🔄 עיבוד כל רשומה
rows = []
for entry in data:
    sku_list = ", ".join([item["sku"] for item in entry.get("items", [])])
    total_qty = sum(item.get("qty", 0) for item in entry.get("items", []))

    row = {
        "FileName": entry.get("source_file_name", ""),
        "Operation": entry.get("operation", ""),
        "Provider": entry.get("provider", ""),
        "TapuzNumber": entry.get("tapuzNumber", ""),
        "Status": entry.get("status", ""),
        "ShipmentStatus": entry.get("tapuzShipmentStatus", ""),
        "TrackingURL": entry.get("urlLink", ""),
        "TargetName": entry.get("targetName", ""),
        "TargetCity": entry.get("targetCity", ""),
        "TargetStreet": entry.get("targetStreetName", ""),
        "Phone": entry.get("targetPhone", ""),
        "Barcode": entry.get("barcode", ""),
        "OrderNumber": entry.get("orderNumber", ""),
        "SKU_List": sku_list,
        "Total_Qty": total_qty
    }
    rows.append(row)

# 📊 יצירת DataFrame
df = pd.DataFrame(rows)

# 💾 שמירה לאקסל
output_file = "tapuz_returns.xlsx"
df.to_excel(output_file, index=False)

# 📥 הורדה
files.download(output_file)
